In [1]:
import sys
import time
import datetime
import os
import tensorflow as tf
import argparse
import json
import socket
from PIL import Image
from util import *
import numpy as np
%load_ext autoreload
%autoreload 2

In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
logPath = "training/"
snapshotPath = "snapshots/"

In [3]:
with open("hyperParams.json") as f:
    instanceParams = json.load(f)

instanceParams['iterations'] = 560000
instanceParams

{u'baseLR': 1e-05,
 u'batchSize': 2,
 u'borderThicknessH': 0.02,
 u'borderThicknessW': 0.02,
 u'boundaryAlpha': 6.5,
 u'dataset': u'kitti2012',
 u'flowScale': 20.0,
 u'gradParams': {u'robustness': 0.46, u'scale': 255.0, u'weight': 6.408},
 u'instanceName': u'unsupFlownet',
 u'iterations': 560000,
 u'lossComponents': {u'asymmetricSmooth': True,
  u'backward': True,
  u'boundaries': False,
  u'gradient': False,
  u'smooth2nd': False},
 u'photoParams': {u'robustness': 0.53, u'scale': 360.0},
 u'printFreq': 1000,
 u'resnet': False,
 u'smooth2ndParams': {u'robustness': 0.21, u'scale': 1.0, u'weight': 0.53},
 u'smoothOccParams': {u'robustness': 0.9, u'scale': 1.8},
 u'smoothParams': {u'robustness': 0.99, u'scale': 3.5, u'weight': 0.05},
 u'snapFreq': 5000,
 u'snapshotFreq': 20000,
 u'weightDecay': 0.0}

In [4]:
printFrequency = instanceParams["printFreq"]
batchSize = instanceParams["batchSize"]

iterations = instanceParams["iterations"]
baseLearningRate = instanceParams["baseLR"]
learningRate = baseLearningRate
snapshotFrequency = instanceParams["snapshotFreq"]

from dotmap import DotMap
arg = DotMap()
arg.logDev = False
arg.resume = 'y'
resume, startIteration, snapshotFiles = checkResume(snapshotPath,logPath, arg)
iterations = 1000 * 100

resuming from iteration 560000


In [5]:
with tf.device("/gpu:0"):
    trainingData = TrainingData(batchSize,instanceParams)

with tf.device("/gpu:0"):
    # init
    with tf.variable_scope("netShare"):
        networkBodyF = NetworkBody(trainingData,instanceParams)
    with tf.variable_scope("netShare",reuse=True):
        networkBodyB = NetworkBody(trainingData,instanceParams,flipInput=True)

    trainingLoss = TrainingLoss(instanceParams,networkBodyF,networkBodyB,trainingData)
    solver,learningRateTensor = attachSolver(trainingLoss.loss)

    # loss scheduling
    recLossBWeightTensor = trainingLoss.recLossBWeight

Reading Tensor("ImagePairData/image_data_reader/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_1/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_2/Placeholder:0", dtype=string, device=/device:GPU:0)
Reading Tensor("ImagePairData/image_data_reader_3/Placeholder:0", dtype=string, device=/device:GPU:0)


Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.


In [6]:
# merge summaries
merged = tf.summary.merge_all()
# saver
saver = tf.train.Saver(max_to_keep=0)

### Training

In [9]:
iterations = 40000

print(printFrequency)
print(snapshotFrequency)
print(iterations)
print(startIteration)

1000
20000
40000
560000


In [ ]:
# start
with sessionSetup(arg) as sess:
    if resume:
        saver.restore(sess,snapshotPath+snapshotFiles[-1][:-6])
    else:
        sess.run(tf.initialize_all_variables())
#     saver.restore(sess,
#                   '../model_download_scripts/photometric_smoothness/weights/iter_0000000000500000.ckpt')

    trainingData.dataQueuer.start_queueing(sess)

    #start summary writer
    summary_writer = tf.summary.FileWriter(logPath, sess.graph)

    #run
    lastPrint = time.time()
    for i in range(startIteration, startIteration + iterations + 10):
        # scheduled values
        learningRate = learningRateSchedule(baseLearningRate, i)
        recLossBWeight = unsupLossBSchedule(i)

         #run training
        feed_dict = {
            learningRateTensor: learningRate,
            recLossBWeightTensor: recLossBWeight,
        }
        summary,result,totalLoss = sess.run([merged,solver,trainingLoss.loss], feed_dict=feed_dict)

        if (i+1) % printFrequency == 0:
            timeDiff = time.time() - lastPrint
            itPerSec = printFrequency/timeDiff
            remainingIt = 500000 + iterations + 10 - i
            eta = remainingIt/itPerSec
            print("Iteration "+str(i+1)+": loss: "+str(totalLoss)+", iterations per second: "+str(itPerSec)+", ETA: "+str(datetime.timedelta(seconds=eta)))+", lr: "+str(learningRate)

            summary_writer.add_summary(summary,i+1)
            summary_writer.flush()
            lastPrint = time.time()

        if (i+1) % snapshotFrequency == 0:
            saver.save(sess,"snapshots/iter_"+str(i+1).zfill(16)+".ckpt")

        sys.stdout.flush()

    #close queing
    trainingData.dataQueuer.close(sess)

INFO:tensorflow:Restoring parameters from snapshots/iter_0000000000560000.ckpt
Iteration 561000: loss: [[14.802177]
 [22.127853]], iterations per second: 0.676596848612, ETA: -1 day, 15:22:58.572627, lr: 2.5e-07
Iteration 562000: loss: [[29.92789 ]
 [20.905231]], iterations per second: 0.690089115244, ETA: -1 day, 15:08:55.999374, lr: 2.5e-07


### Testing

In [ ]:
datasetRoot = '../example_data/'
frame0Path = '../example_data/datalists/train_im0.txt'
frame1Path = '../example_data/datalists/train_im1.txt'
desiredHeight = 480
desiredWidth = 854

In [ ]:
with open(frame0Path) as f:
    imagePairs0 = [datasetRoot+x[:-1] for x in f.readlines()]
with open(frame1Path) as f:
    imagePairs1 = [datasetRoot+x[:-1] for x in f.readlines()]

In [ ]:
testData = TestData(imagePairs0,imagePairs1,1,desiredHeight,desiredWidth)

In [ ]:
with tf.device("/gpu:0"):
    with tf.variable_scope("netShare"):
        networkBody = NetworkBody(testData,instanceParams)
flowFinal = networkBody.flows[0]

In [ ]:
flowViz = flowToRgb(flowFinal)
transformGrid = flowTransformGrid(flowFinal)
mean = [0.407871, 0.457525, 0.481094]
mean = tf.expand_dims(tf.expand_dims(tf.expand_dims(mean,0),0),0)
# warped = flowWarp(testData.frame1["rgb"]+mean,flowFinal)

In [ ]:
saver = tf.train.Saver()

# config tensorflow
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
config.allow_soft_placement = True
iterations = 510000

In [ ]:
start = None
for i in range(len(imagePairs0)):
    if 'hike' in imagePairs0[i]:
        start = i
        break

In [ ]:
test_length = 70
start = 0
result_dir = 'results'
with tf.Session(config=config) as sess:
    saver.restore(sess,"snapshots_orig/iter_"+str(iterations).zfill(16)+".ckpt")

    # run
    lastPrint = time.time()
    for i in range(start, start + test_length):
        feed_dict = {
            testData.im0File: imagePairs0[i],
            testData.im1File: imagePairs1[i],
        }
        result = sess.run([flowViz,testData.height,testData.width],feed_dict=feed_dict)
        h = result[1]
        w = result[2]
        arr = np.maximum(np.minimum(np.asarray(result[0]),1), 0)
        arr = np.squeeze(np.asarray(arr*255,np.uint8))
        im = Image.fromarray(arr[:h,:w,:])
        im.save("{}/{}.png".format(result_dir, str(i).zfill(3)))